### Import Dependencies

In [5]:
import sys
sys.path.append("../")
import pandas as pd

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model,should_use_reasoning_model
from IPython.display import display, Markdown

### Loading data from sample.txt file

In [6]:
file_path = '../data/raw/Scenarios.txt'

with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()

# Split scenarios by blank lines
scenarios = [s.strip() for s in text.split("\n\n") if s.strip()]

print(f"Loaded {len(scenarios)} scenarios\n")

Loaded 2 scenarios



### LLM Client

In [7]:
model = pick_model('openai', 'cot')
client = LLMClient('openai', model)

instruction = """
Analyze the following crisis scenario using a strict, step-by-step reasoning process.

Follow these rules exactly:
1. Identify all immediate life-threatening situations.
2. Identify all immediate health-threatening situations.
3. Identify secondary risks (logistics, hunger, infrastructure).
4. Prioritize actions using this fixed order:
   a. Immediate life threats
   b. Immediate health threats
   c. Secondary risks
5. If resources are limited, allocate them to the highest-priority category first.
6. Do not change the priority order under any circumstances.
7. Provide a final ordered list of actions based on the above logic.
8. Do not add creative alternatives or speculative assumptions.
"""

prompt_text, spec = render(
    'cot_reasoning.v1',
    role='crisis_response_analyst',
    problem="{scenario}"
)

print(prompt_text) 




You are crisis_response_analyst. Apply the specified crisis-prioritization rules exactly.
Problem: {scenario}

First, outline your reasoning steps briefly.
Then provide the final answer clearly marked under 'Answer:'.
Keep reasoning concise; avoid unnecessary prose.



In [8]:
for i, scenario in enumerate(scenarios, 1):
    full_prompt = f"""text: {prompt_text.replace("{scenario}", scenario)}
instruction: {instruction}"""

    messages = [
        {"role": "user", "content": full_prompt}
    ]

    print(f"\n=== Scenario {i} ===")
    print(scenario)
    print("\n--- Chaos Mode Responses ---")

    # Run 3 times (Chaos Mode)
    for j in range(1, 4):
        response = client.chat(messages, temperature=1.0, max_tokens=spec.max_tokens)
        print(f"\n[Chaos Mode Run {j}]\n")
        print(response['text'])
        print("\n" + "-"*50)
    
    # Optional: Safe Mode
    response_safe = client.chat(messages, temperature=0.0, max_tokens=spec.max_tokens)
    print("\n--- Safe Mode Response ---\n")
    print(response_safe['text'])
    print("="*80 + "\n")


=== Scenario 1 ===
SCENARIO A: THE KANDY LANDSLIDE
Location: Hanthana Tea Factory
Details: "We are trapped. The access road is blocked. My uncle is stuck in the line rooms below, climbing a tree as water rises (Immediate Life Threat). However, inside the factory, we have a diabetic patient who has collapsed and needs insulin (Immediate Health Threat). We also have 40 people hungry."

--- Chaos Mode Responses ---

[Chaos Mode Run 1]

Reasoning:
1. Immediate life threat: The uncle stuck below in a rising water situation.
2. Immediate health threat: The collapsed diabetic patient who needs insulin.
3. Secondary risk: The hunger of 40 people.
4. Prioritized actions:
 a. Rescue the uncle.
 b. Provide urgent medical care (insulin) to the diabetic patient.
 c. Arrange food support for the 40 people.

Answer:
1. Immediate Life Threat: Rescue the trapped uncle in the line rooms.
2. Immediate Health Threat: Administer insulin and provide urgent care to the collapsed diabetic patient.
3. Seconda

# Deliverable

# 1️⃣ Observations on Scenario 1 (Kandy Landslide)

## Safe Mode
- **Prioritization:** Always follows exactly: **life threat → health threat → secondary risk**.
- **Consistency:** Wording is consistent, concise, and deterministic.
- **Clarity:** Step-by-step actions are clearly linked to threats.
- **No speculative actions:** Safe Mode does not introduce anything beyond the scenario.

## Chaos Mode
- **Variations in wording:** Phrases like `"mobilize resources"` vs `"arrange food support"` vs `"once immediate threats are managed, restore access"` appear across runs.
- **Minor drift / extra context:** Sometimes introduces context not strictly in the original scenario (e.g., `"restore access road"` is mentioned in some Chaos runs but not in Safe Mode).
- **Prioritization:** The order **remains correct**, but the focus and level of detail can drift slightly depending on the run.

### Drift / Hallucination Examples
- **[Chaos Mode Run 2]:** Adds `"restore access road as soon as possible"` → not explicitly in scenario, mild hallucination.
- **[Chaos Mode Run 3]:** Merges **hunger** and **blocked access** together, whereas Safe Mode keeps them separate.


# 2️⃣ Observations on Scenario 2 (Gampaha Hospital)

## Safe Mode
- **Prioritization:** Strictly follows **life threat → health threat → secondary risks**.
- **Consistency:** Deterministic wording, concise, and step-by-step.
- **Clarity:** Every action clearly corresponds to identified threats.
- **No speculation:** Only addresses what is explicitly in the scenario.

## Chaos Mode
- **Variations in wording:** Different runs use phrases like `"mobilize water barriers, drainage"` vs `"deploy sandbags, call for additional flood response"` vs `"initiate water diversion or pumping efforts"`.
- **Minor drift / extra context:** Occasionally introduces additional suggestions not explicitly in the scenario.
- **Prioritization:** The ICU life threat is consistently prioritized first, followed by the flood-affected ward, then secondary risks.

### Drift / Hallucination Examples
- **[Chaos Mode Run 1]:** Mentions `"monitor both situations continuously"` → extra detail not in Safe Mode.
- **[Chaos Mode Run 3]:** Adds `"longer-term patient safety"` → minor speculative addition beyond the scenario.
